In [1]:
%load_ext autoreload
%autoreload 2

import sys
# change this to suit your path
sys.path.append('/home/guillaume/Python/kt_simul/')
from kt_simul.smp_spindle.smp_spindle import SympySpindle, parameters


import IPython.display as disp
import os

### Visualisation using openscad

This has the advangtage of being simple to use

```python
import solid as solid
import tempfile
solid.use('spindle.scad')
spindle_length = 4
ch_pos = [-0.4, 0.1, 1]
stretch = 0.3
ch_theta = 30
ch_phi = 10

scad_spindle = (spindle_midzone(spindle_length) +
                chromosome(ch_pos, stretch, ch_theta, ch_phi))
def show_scad(scad_spindle, img_name='spindle_scad.png'):
    if os.path.isfile(img_name):
        os.remove(img_name)
    with tempfile.NamedTemporaryFile('w', suffix='.scad') as tmp_scad:
        solid.scad_render_to_file(scad_spindle, tmp_scad.name)
        os.system('openscad -o {} {}'.format(img_name, tmp_scad.name))
    return disp.Image(img_name)
show_scad(scad_spindle)
```

In [2]:
from sympy.physics.mechanics import mechanics_printing

mechanics_printing(pretty_print=False)  # Shinny


In [3]:
import vispy.geometry as vp_geom
import vispy.util as vp_util

def translate(mesh, pos):
    
    mesh.set_vertices(mesh.get_vertices() + pos)
    return mesh
    

def create_spb(pos, radius=0.3):
    spb = vp_geom.create_sphere(12, 12, radius)
    return translate(spb, pos)
    
    
def create_midzone(length, r=0.2):
    
    mz = vp_geom.create_cylinder(12, 2, [r, r],
                                 length, True)
    return mz
    
def create_chromosome(stretch, orientation, r=0.1):
    
    theta, phi, omega = orientation
    
    ch = vp_geom.create_cylinder(12, 2, [r, r],
                                 length, True)
    
### Maybe scene visuals is better suited, see:
### http://api.vispy.org/en/latest/scene.html

from vispy import app, gloo, visuals
from vispy.visuals.transforms import (STTransform, AffineTransform,
                                      ChainTransform)


class Canvas(app.Canvas):
    def __init__(self, mdatas):
        app.Canvas.__init__(self, keys='interactive', size=(800, 550))

        self.rotation = AffineTransform()
        self.meshes = [visuals.MeshVisual(meshdata=mdata, color='r') for mdata in mdatas]

        self.show()

        self.timer = app.Timer(connect=self.rotate)
        self.timer.start(0.016)

    def rotate(self, event):
        self.rotation.rotate(1, (0, 1, 0))
        self.update()

    def on_draw(self, ev):
        gloo.set_viewport(0, 0, *self.physical_size)
        gloo.clear(color='black', depth=True)
        for mesh in self.meshes:
            mesh.draw()

length = 3.

            
spbL = create_spb([-length/2, 0, 0])
spbR = create_spb([length/2, 0, 0])
mz = create_midzone(length)

#win = Canvas([spbL, spbR, mz])
#import sys a
#if sys.flags.interactive != 1:
#    app.run()

### Too be continued    



ImportError: cannot import name 'AffineTransform'

### Model description

We're looking at a simple mechanical system

![3D spindle](3d_spindle.svg)



#### Points and coordiantes

Point is the physical location of an element.

In the method we use to write the equations of motion of the system, we use what is caled **generalized coordinates** and **generalized speeds**. The generic letter for a gen. coordinate is **q** and **u** for the speeds. Those are the quantities marked in red on the sketch.

As a starting point to a 3D description of the spindle, we assume that the chromosomes are 'aligned', thus there is a single extra reference frame, noted $\mathbf{C}$, for each chromosome.




### Parameters 

In [4]:
## Those are simple  `sympy.symbols`
for param in parameters.keys():
    print(param)

d_0
mu_c
F_mz
kappa_k
kappa_c
mu_k
V_k
mu_s
F_k
mu_ch
V_mz


## Spindle class instanciation



In [5]:
spindle = SympySpindle('3D_1_1', N=1, Mk=1)

In [6]:
ch = spindle.chromosomes[0]

In [7]:
ps = ch.cen_A.plugsites[0]

In [8]:
cen_A00 = spindle.points[3]

In [30]:
a = np.zeros((5, 3))

a[0] = (1, 1, 1)

In [31]:
a

array([[ 1.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [9]:
spindle.kanesmethod()

In [10]:
spindle.attach_state

Matrix([
[lambda_00A, rho_00A],
[lambda_00B, rho_00B]])

In [11]:
import sympy as sy
from sympy import sympify, lambdify


In [12]:
amphitelic = {astate:val for astate, val in zip(spindle.attach_state, sy.Identity(2))}

In [13]:
passive_params = ['mu_s', 'mu_ch', 'mu_c','mu_k',
                  'kappa_c', 'kappa_k', 'd_0']
passive_zero = {parameters[name]:0 for name in passive_params}

active_params = ['F_k', 'V_k', 'F_mz', 'V_mz']
active_zero = {parameters[name]:0 for name in active_params}

adim = {parameters['F_k']:1, parameters['V_k']:1}

In [14]:
A0 = spindle.A_uu.subs(active_zero).subs(adim)
At = spindle.A_uu.subs(passive_zero).subs(adim)
At_args = [parameters[name] for name in active_params] + list(spindle.attach_state)
A0_args = [parameters[name] for name in passive_params]

In [15]:
A0

Matrix([
[-mu_s/2,                    0,                    0,                    0,     0,     0,     0,                    0,                    0,                    0,     0,     0,     0],
[      0, -mu_c - mu_ch - mu_k,                    0,                    0,  mu_k,     0,     0,                 mu_c,                    0,                    0,     0,     0,     0],
[      0,                    0, -mu_c - mu_ch - mu_k,                    0,     0,  mu_k,     0,                    0,                 mu_c,                    0,     0,     0,     0],
[      0,                    0,                    0, -mu_c - mu_ch - mu_k,     0,     0,  mu_k,                    0,                    0,                 mu_c,     0,     0,     0],
[      0,                 mu_k,                    0,                    0, -mu_k,     0,     0,                    0,                    0,                    0,     0,     0,     0],
[      0,                    0,                 mu_k,             

In [16]:
At_args

[F_k, V_k, F_mz, V_mz, lambda_00A, rho_00A, lambda_00B, rho_00B]

In [17]:
spindle.q_ind

[L, x_0A, y_0A, z_0A, x_00A, y_00A, z_00A, x_0B, y_0B, z_0B, x_00B, y_00B, z_00B]

In [18]:
At_np = lambdify(At_args, At, 'numpy', dummify=False)
A0_np = lambdify(A0_args, A0, 'numpy', dummify=False)

In [21]:
import numpy as np

In [24]:
np.linalg.norm(np.array((1, 2, 3)) - np.array((4, 5, 6)))

5.19615242271

In [44]:
A0_np()

<function numpy.<lambda>>

In [28]:
At_np(1, 1, 5, 1, 1, 0, 1, 0)

matrix([[-5.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,
          0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ],
        [-0.5,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ],
        [-0.5,  0. ,  0. ,  0.

In [36]:
spindle.q_ind

[L, x_0A, y_0A, z_0A, x_00A, y_00A, z_00A, x_0B, y_0B, z_0B, x_00B, y_00B, z_00B]

In [35]:
spindle.KM.forcelist[-1][1]

- mu_c*(-vx_0A + vx_0B)*S.x - mu_c*(-vy_0A + vy_0B)*S.y - mu_c*(-vz_0A + vz_0B)*S.z

In [37]:
A0

Matrix([
[-mu_s/2,                    0,                    0,                    0,     0,     0,     0,                    0,                    0,                    0,     0,     0,     0],
[      0, -mu_c - mu_ch - mu_k,                    0,                    0,  mu_k,     0,     0,                 mu_c,                    0,                    0,     0,     0,     0],
[      0,                    0, -mu_c - mu_ch - mu_k,                    0,     0,  mu_k,     0,                    0,                 mu_c,                    0,     0,     0,     0],
[      0,                    0,                    0, -mu_c - mu_ch - mu_k,     0,     0,  mu_k,                    0,                    0,                 mu_c,     0,     0,     0],
[      0,                 mu_k,                    0,                    0, -mu_k,     0,     0,                    0,                    0,                    0,     0,     0,     0],
[      0,                    0,                 mu_k,             

In [43]:
from kt_simul.io.parameters_io import ParamTree
from kt_simul.core.simul_spindle import Metaphase